In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
# import torch.nn.functional as F
import glob
import scipy.io as sio
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import torchvision.transforms as transforms
from PIL import Image
import vgg
import glob
import numpy as np

import REGroup 


In [2]:
device = 'cuda'
PATH = './vgg19_cifar10_trained_model.pth.tar'
model = vgg.__dict__['vgg19']()
model.features = torch.nn.DataParallel(model.features)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
model = model.to(device)
model.eval()

VGG(
  (features): DataParallel(
    (module): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      

In [3]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])

gc_path = "./layerwise_generative_classifiers/"
RG_model = REGroup.REGroup(eval_model=model,gc_path=gc_path,device=device)

In [4]:
# SoftMax vs REGroup Clean Sample Accuracy

softmax_correct_pred = 0
REGroup_correct_pred = 0
tot_samples = 0
model.eval()
for cid in range(10):
    
    samples = glob.glob('./cifar10_test_set/'+str(cid)+'/*.mat')
    tot_samples = tot_samples+len(samples)
    for i in range(len(samples)):
        print("Evaluating Class:{} samples:{}".format(cid,i))
        data = sio.loadmat(samples[i])
        img =  torch.tensor(data['img']).permute(1,2,0).numpy()
        input = Image.fromarray(img)
        input = transform(input).unsqueeze(0).to(device)
        label = torch.tensor(data['label'])[0][0].to(device)
        
        #---------SoftMax Prediction--------------
        with torch.no_grad():
            output = model(input)

        smax = torch.softmax(output,dim=1)
        pred_class = torch.argmax(smax)

        if pred_class == label:
            softmax_correct_pred = softmax_correct_pred+1
        #------------------------------------------
        #--------REGroup Prediction----------------
        REGroup_ranked_predictions=RG_model.get_robust_predictions(img=input)
        Regroup_pred_class = REGroup_ranked_predictions[0]   #Top-1 Class

        if Regroup_pred_class == label:
            REGroup_correct_pred = REGroup_correct_pred+1
        #------------------------------------------
        
SoftMax_CIFAR10_Test_Clean_Samples_ACC = (softmax_correct_pred/tot_samples)*100
REGroup_CIFAR10_Test_Clean_Samples_ACC = (REGroup_correct_pred/tot_samples)*100

print('Clean Samples Accuracy. PreTrained+SoftMax:{}%, PreTrained+REGroup:{}%'.format(SoftMax_CIFAR10_Test_Clean_Samples_ACC,
                                                                                  REGroup_CIFAR10_Test_Clean_Samples_ACC ))

Evaluating Class:0 samples:0
Evaluating Class:0 samples:1
Evaluating Class:0 samples:2
Evaluating Class:0 samples:3
Evaluating Class:0 samples:4
Evaluating Class:0 samples:5
Evaluating Class:0 samples:6
Evaluating Class:0 samples:7
Evaluating Class:0 samples:8
Evaluating Class:0 samples:9
Evaluating Class:0 samples:10
Evaluating Class:0 samples:11
Evaluating Class:0 samples:12
Evaluating Class:0 samples:13
Evaluating Class:0 samples:14
Evaluating Class:0 samples:15
Evaluating Class:0 samples:16
Evaluating Class:0 samples:17
Evaluating Class:0 samples:18
Evaluating Class:0 samples:19
Evaluating Class:0 samples:20
Evaluating Class:0 samples:21
Evaluating Class:0 samples:22
Evaluating Class:0 samples:23
Evaluating Class:0 samples:24
Evaluating Class:0 samples:25
Evaluating Class:0 samples:26
Evaluating Class:0 samples:27
Evaluating Class:0 samples:28
Evaluating Class:0 samples:29
Evaluating Class:0 samples:30
Evaluating Class:0 samples:31
Evaluating Class:0 samples:32
Evaluating Class:0 s

Evaluating Class:0 samples:269
Evaluating Class:0 samples:270
Evaluating Class:0 samples:271
Evaluating Class:0 samples:272
Evaluating Class:0 samples:273
Evaluating Class:0 samples:274
Evaluating Class:0 samples:275
Evaluating Class:0 samples:276
Evaluating Class:0 samples:277
Evaluating Class:0 samples:278
Evaluating Class:0 samples:279
Evaluating Class:0 samples:280
Evaluating Class:0 samples:281
Evaluating Class:0 samples:282
Evaluating Class:0 samples:283
Evaluating Class:0 samples:284
Evaluating Class:0 samples:285
Evaluating Class:0 samples:286
Evaluating Class:0 samples:287
Evaluating Class:0 samples:288
Evaluating Class:0 samples:289
Evaluating Class:0 samples:290
Evaluating Class:0 samples:291
Evaluating Class:0 samples:292
Evaluating Class:0 samples:293
Evaluating Class:0 samples:294
Evaluating Class:0 samples:295
Evaluating Class:0 samples:296
Evaluating Class:0 samples:297
Evaluating Class:0 samples:298
Evaluating Class:0 samples:299
Evaluating Class:0 samples:300
Evaluati

Evaluating Class:0 samples:537
Evaluating Class:0 samples:538
Evaluating Class:0 samples:539
Evaluating Class:0 samples:540
Evaluating Class:0 samples:541
Evaluating Class:0 samples:542
Evaluating Class:0 samples:543
Evaluating Class:0 samples:544
Evaluating Class:0 samples:545
Evaluating Class:0 samples:546
Evaluating Class:0 samples:547
Evaluating Class:0 samples:548
Evaluating Class:0 samples:549
Evaluating Class:0 samples:550
Evaluating Class:0 samples:551
Evaluating Class:0 samples:552
Evaluating Class:0 samples:553
Evaluating Class:0 samples:554
Evaluating Class:0 samples:555
Evaluating Class:0 samples:556
Evaluating Class:0 samples:557
Evaluating Class:0 samples:558
Evaluating Class:0 samples:559
Evaluating Class:0 samples:560
Evaluating Class:0 samples:561
Evaluating Class:0 samples:562
Evaluating Class:0 samples:563
Evaluating Class:0 samples:564
Evaluating Class:0 samples:565
Evaluating Class:0 samples:566
Evaluating Class:0 samples:567
Evaluating Class:0 samples:568
Evaluati

Evaluating Class:0 samples:805
Evaluating Class:0 samples:806
Evaluating Class:0 samples:807
Evaluating Class:0 samples:808
Evaluating Class:0 samples:809
Evaluating Class:0 samples:810
Evaluating Class:0 samples:811
Evaluating Class:0 samples:812
Evaluating Class:0 samples:813
Evaluating Class:0 samples:814
Evaluating Class:0 samples:815
Evaluating Class:0 samples:816
Evaluating Class:0 samples:817
Evaluating Class:0 samples:818
Evaluating Class:0 samples:819
Evaluating Class:0 samples:820
Evaluating Class:0 samples:821
Evaluating Class:0 samples:822
Evaluating Class:0 samples:823
Evaluating Class:0 samples:824
Evaluating Class:0 samples:825
Evaluating Class:0 samples:826
Evaluating Class:0 samples:827
Evaluating Class:0 samples:828
Evaluating Class:0 samples:829
Evaluating Class:0 samples:830
Evaluating Class:0 samples:831
Evaluating Class:0 samples:832
Evaluating Class:0 samples:833
Evaluating Class:0 samples:834
Evaluating Class:0 samples:835
Evaluating Class:0 samples:836
Evaluati

Evaluating Class:1 samples:73
Evaluating Class:1 samples:74
Evaluating Class:1 samples:75
Evaluating Class:1 samples:76
Evaluating Class:1 samples:77
Evaluating Class:1 samples:78
Evaluating Class:1 samples:79
Evaluating Class:1 samples:80
Evaluating Class:1 samples:81
Evaluating Class:1 samples:82
Evaluating Class:1 samples:83
Evaluating Class:1 samples:84
Evaluating Class:1 samples:85
Evaluating Class:1 samples:86
Evaluating Class:1 samples:87
Evaluating Class:1 samples:88
Evaluating Class:1 samples:89
Evaluating Class:1 samples:90
Evaluating Class:1 samples:91
Evaluating Class:1 samples:92
Evaluating Class:1 samples:93
Evaluating Class:1 samples:94
Evaluating Class:1 samples:95
Evaluating Class:1 samples:96
Evaluating Class:1 samples:97
Evaluating Class:1 samples:98
Evaluating Class:1 samples:99
Evaluating Class:1 samples:100
Evaluating Class:1 samples:101
Evaluating Class:1 samples:102
Evaluating Class:1 samples:103
Evaluating Class:1 samples:104
Evaluating Class:1 samples:105
Eval

...

Evaluating Class:8 samples:845
Evaluating Class:8 samples:846
Evaluating Class:8 samples:847
Evaluating Class:8 samples:848
Evaluating Class:8 samples:849
Evaluating Class:8 samples:850
Evaluating Class:8 samples:851
Evaluating Class:8 samples:852
Evaluating Class:8 samples:853
Evaluating Class:8 samples:854
Evaluating Class:8 samples:855
Evaluating Class:8 samples:856
Evaluating Class:8 samples:857
Evaluating Class:8 samples:858
Evaluating Class:8 samples:859
Evaluating Class:8 samples:860
Evaluating Class:8 samples:861
Evaluating Class:8 samples:862
Evaluating Class:8 samples:863
Evaluating Class:8 samples:864
Evaluating Class:8 samples:865
Evaluating Class:8 samples:866
Evaluating Class:8 samples:867
Evaluating Class:8 samples:868
Evaluating Class:8 samples:869
Evaluating Class:8 samples:870
Evaluating Class:8 samples:871
Evaluating Class:8 samples:872
Evaluating Class:8 samples:873
Evaluating Class:8 samples:874
Evaluating Class:8 samples:875
Evaluating Class:8 samples:876
Evaluati

Evaluating Class:9 samples:113
Evaluating Class:9 samples:114
Evaluating Class:9 samples:115
Evaluating Class:9 samples:116
Evaluating Class:9 samples:117
Evaluating Class:9 samples:118
Evaluating Class:9 samples:119
Evaluating Class:9 samples:120
Evaluating Class:9 samples:121
Evaluating Class:9 samples:122
Evaluating Class:9 samples:123
Evaluating Class:9 samples:124
Evaluating Class:9 samples:125
Evaluating Class:9 samples:126
Evaluating Class:9 samples:127
Evaluating Class:9 samples:128
Evaluating Class:9 samples:129
Evaluating Class:9 samples:130
Evaluating Class:9 samples:131
Evaluating Class:9 samples:132
Evaluating Class:9 samples:133
Evaluating Class:9 samples:134
Evaluating Class:9 samples:135
Evaluating Class:9 samples:136
Evaluating Class:9 samples:137
Evaluating Class:9 samples:138
Evaluating Class:9 samples:139
Evaluating Class:9 samples:140
Evaluating Class:9 samples:141
Evaluating Class:9 samples:142
Evaluating Class:9 samples:143
Evaluating Class:9 samples:144
Evaluati

Evaluating Class:9 samples:381
Evaluating Class:9 samples:382
Evaluating Class:9 samples:383
Evaluating Class:9 samples:384
Evaluating Class:9 samples:385
Evaluating Class:9 samples:386
Evaluating Class:9 samples:387
Evaluating Class:9 samples:388
Evaluating Class:9 samples:389
Evaluating Class:9 samples:390
Evaluating Class:9 samples:391
Evaluating Class:9 samples:392
Evaluating Class:9 samples:393
Evaluating Class:9 samples:394
Evaluating Class:9 samples:395
Evaluating Class:9 samples:396
Evaluating Class:9 samples:397
Evaluating Class:9 samples:398
Evaluating Class:9 samples:399
Evaluating Class:9 samples:400
Evaluating Class:9 samples:401
Evaluating Class:9 samples:402
Evaluating Class:9 samples:403
Evaluating Class:9 samples:404
Evaluating Class:9 samples:405
Evaluating Class:9 samples:406
Evaluating Class:9 samples:407
Evaluating Class:9 samples:408
Evaluating Class:9 samples:409
Evaluating Class:9 samples:410
Evaluating Class:9 samples:411
Evaluating Class:9 samples:412
Evaluati

Evaluating Class:9 samples:649
Evaluating Class:9 samples:650
Evaluating Class:9 samples:651
Evaluating Class:9 samples:652
Evaluating Class:9 samples:653
Evaluating Class:9 samples:654
Evaluating Class:9 samples:655
Evaluating Class:9 samples:656
Evaluating Class:9 samples:657
Evaluating Class:9 samples:658
Evaluating Class:9 samples:659
Evaluating Class:9 samples:660
Evaluating Class:9 samples:661
Evaluating Class:9 samples:662
Evaluating Class:9 samples:663
Evaluating Class:9 samples:664
Evaluating Class:9 samples:665
Evaluating Class:9 samples:666
Evaluating Class:9 samples:667
Evaluating Class:9 samples:668
Evaluating Class:9 samples:669
Evaluating Class:9 samples:670
Evaluating Class:9 samples:671
Evaluating Class:9 samples:672
Evaluating Class:9 samples:673
Evaluating Class:9 samples:674
Evaluating Class:9 samples:675
Evaluating Class:9 samples:676
Evaluating Class:9 samples:677
Evaluating Class:9 samples:678
Evaluating Class:9 samples:679
Evaluating Class:9 samples:680
Evaluati

Evaluating Class:9 samples:917
Evaluating Class:9 samples:918
Evaluating Class:9 samples:919
Evaluating Class:9 samples:920
Evaluating Class:9 samples:921
Evaluating Class:9 samples:922
Evaluating Class:9 samples:923
Evaluating Class:9 samples:924
Evaluating Class:9 samples:925
Evaluating Class:9 samples:926
Evaluating Class:9 samples:927
Evaluating Class:9 samples:928
Evaluating Class:9 samples:929
Evaluating Class:9 samples:930
Evaluating Class:9 samples:931
Evaluating Class:9 samples:932
Evaluating Class:9 samples:933
Evaluating Class:9 samples:934
Evaluating Class:9 samples:935
Evaluating Class:9 samples:936
Evaluating Class:9 samples:937
Evaluating Class:9 samples:938
Evaluating Class:9 samples:939
Evaluating Class:9 samples:940
Evaluating Class:9 samples:941
Evaluating Class:9 samples:942
Evaluating Class:9 samples:943
Evaluating Class:9 samples:944
Evaluating Class:9 samples:945
Evaluating Class:9 samples:946
Evaluating Class:9 samples:947
Evaluating Class:9 samples:948
Evaluati

In [5]:
# SoftMax vs REGroup Accuracy (CIFAR10 Test Set) on Adversarial Samples generated using PGD with L-Infinity perturbation Metric
#------------------------------------------------------
# We have pre generated the pgd examples with L-infinity metric
# If you would like to generate on your own please check next cell.


data = sio.loadmat('cifar10_vgg19_pgd_examples.mat')
pgd_linf_cifar10_samples = data['adv_ex']
all_effective_dist =  data['all_effective_dist']
gt_label = sio.loadmat('lbl.mat')['lbl']
sam_id=0
adv_softmax_correct_pred = 0
adv_REGroup_correct_pred = 0
tot_samples = 0
all_effective_dist=[]
model.eval()
for sid in range(0,10000,1):
        print('Evaluating sample {}'.format(sid))
        adv_img =torch.tensor(pgd_linf_cifar10_samples[sid,:,:,:].astype(np.float32)).unsqueeze(0)
        # Adversarial images (adv_img) are already normalized using mean and std, DO NOT APPLY normlaization again
        label = gt_label[0,sid]
        #---------Adversarial Sample SoftMax Prediction--------------
        with torch.no_grad():
            adv_output = model(adv_img)

        adv_smax = torch.softmax(adv_output,dim=1)
        adv_pred_class = torch.argmax(adv_smax)
        
        if adv_pred_class == label:
            adv_softmax_correct_pred = adv_softmax_correct_pred+1
            
        
        #--------Adversarial Sample REGroup Prediction----------------
        adv_REGroup_ranked_predictions=RG_model.get_robust_predictions(img=adv_img)
        adv_Regroup_pred_class = adv_REGroup_ranked_predictions[0]   #Top-1 Class

        if adv_Regroup_pred_class == label:
            adv_REGroup_correct_pred = adv_REGroup_correct_pred+1
        #------------------------------------------

SoftMax_CIFAR10_Test_Adv_Samples_ACC = (adv_softmax_correct_pred/10000.0)*100
REGroup_CIFAR10_Test_Adv_Samples_ACC = (adv_REGroup_correct_pred/10000.0)*100

print('CIFAR10 Test set average PGD Linfinity perturbation epsilon {}/256'.format(np.round(np.mean(data['all_effective_dist']))))
print('Adversarial Samples Accuracy. PreTrained+SoftMax:{}%, PreTrained+REGroup:{}%'.format(SoftMax_CIFAR10_Test_Adv_Samples_ACC,
                                                                                  REGroup_CIFAR10_Test_Adv_Samples_ACC ))

Evaluating sample 0
Evaluating sample 1
Evaluating sample 2
Evaluating sample 3
Evaluating sample 4
Evaluating sample 5
Evaluating sample 6
Evaluating sample 7
Evaluating sample 8
Evaluating sample 9
Evaluating sample 10
Evaluating sample 11
Evaluating sample 12
Evaluating sample 13
Evaluating sample 14
Evaluating sample 15
Evaluating sample 16
Evaluating sample 17
Evaluating sample 18
Evaluating sample 19
Evaluating sample 20
Evaluating sample 21
Evaluating sample 22
Evaluating sample 23
Evaluating sample 24
Evaluating sample 25
Evaluating sample 26
Evaluating sample 27
Evaluating sample 28
Evaluating sample 29
Evaluating sample 30
Evaluating sample 31
Evaluating sample 32
Evaluating sample 33
Evaluating sample 34
Evaluating sample 35
Evaluating sample 36
Evaluating sample 37
Evaluating sample 38
Evaluating sample 39
Evaluating sample 40
Evaluating sample 41
Evaluating sample 42
Evaluating sample 43
Evaluating sample 44
Evaluating sample 45
Evaluating sample 46
Evaluating sample 47
Ev

Evaluating sample 380
Evaluating sample 381
Evaluating sample 382
Evaluating sample 383
Evaluating sample 384
Evaluating sample 385
Evaluating sample 386
Evaluating sample 387
Evaluating sample 388
Evaluating sample 389
Evaluating sample 390
Evaluating sample 391
Evaluating sample 392
Evaluating sample 393
Evaluating sample 394
Evaluating sample 395
Evaluating sample 396
Evaluating sample 397
Evaluating sample 398
Evaluating sample 399
Evaluating sample 400
Evaluating sample 401
Evaluating sample 402
Evaluating sample 403
Evaluating sample 404
Evaluating sample 405
Evaluating sample 406
Evaluating sample 407
Evaluating sample 408
Evaluating sample 409
Evaluating sample 410
Evaluating sample 411
Evaluating sample 412
Evaluating sample 413
Evaluating sample 414
Evaluating sample 415
Evaluating sample 416
Evaluating sample 417
Evaluating sample 418
Evaluating sample 419
Evaluating sample 420
Evaluating sample 421
Evaluating sample 422
Evaluating sample 423
Evaluating sample 424
Evaluating

Evaluating sample 756
Evaluating sample 757
Evaluating sample 758
Evaluating sample 759
Evaluating sample 760
Evaluating sample 761
Evaluating sample 762
Evaluating sample 763
Evaluating sample 764
Evaluating sample 765
Evaluating sample 766
Evaluating sample 767
Evaluating sample 768
Evaluating sample 769
Evaluating sample 770
Evaluating sample 771
Evaluating sample 772
Evaluating sample 773
Evaluating sample 774
Evaluating sample 775
Evaluating sample 776
Evaluating sample 777
Evaluating sample 778
Evaluating sample 779
Evaluating sample 780
Evaluating sample 781
Evaluating sample 782
Evaluating sample 783
Evaluating sample 784
Evaluating sample 785
Evaluating sample 786
Evaluating sample 787
Evaluating sample 788
Evaluating sample 789
Evaluating sample 790
Evaluating sample 791
Evaluating sample 792
Evaluating sample 793
Evaluating sample 794
Evaluating sample 795
Evaluating sample 796
Evaluating sample 797
Evaluating sample 798
Evaluating sample 799
Evaluating sample 800
Evaluating

Evaluating sample 1124
Evaluating sample 1125
Evaluating sample 1126
Evaluating sample 1127
Evaluating sample 1128
Evaluating sample 1129
Evaluating sample 1130
Evaluating sample 1131
Evaluating sample 1132
Evaluating sample 1133
Evaluating sample 1134
Evaluating sample 1135
Evaluating sample 1136
Evaluating sample 1137
Evaluating sample 1138
Evaluating sample 1139
Evaluating sample 1140
Evaluating sample 1141
Evaluating sample 1142
Evaluating sample 1143
Evaluating sample 1144
Evaluating sample 1145
Evaluating sample 1146
Evaluating sample 1147
Evaluating sample 1148
Evaluating sample 1149
Evaluating sample 1150
Evaluating sample 1151
Evaluating sample 1152
Evaluating sample 1153
Evaluating sample 1154
Evaluating sample 1155
Evaluating sample 1156
Evaluating sample 1157
Evaluating sample 1158
Evaluating sample 1159
Evaluating sample 1160
Evaluating sample 1161
Evaluating sample 1162
Evaluating sample 1163
Evaluating sample 1164
Evaluating sample 1165
Evaluating sample 1166
Evaluating 

Evaluating sample 1484
Evaluating sample 1485
Evaluating sample 1486
Evaluating sample 1487
Evaluating sample 1488
Evaluating sample 1489
Evaluating sample 1490
Evaluating sample 1491
Evaluating sample 1492
Evaluating sample 1493
Evaluating sample 1494
Evaluating sample 1495
Evaluating sample 1496
Evaluating sample 1497
Evaluating sample 1498
Evaluating sample 1499
Evaluating sample 1500
Evaluating sample 1501
Evaluating sample 1502
Evaluating sample 1503
Evaluating sample 1504
Evaluating sample 1505
Evaluating sample 1506
Evaluating sample 1507
Evaluating sample 1508
Evaluating sample 1509
Evaluating sample 1510
Evaluating sample 1511
Evaluating sample 1512
Evaluating sample 1513
Evaluating sample 1514
Evaluating sample 1515
Evaluating sample 1516
Evaluating sample 1517
Evaluating sample 1518
Evaluating sample 1519
Evaluating sample 1520
Evaluating sample 1521
Evaluating sample 1522
Evaluating sample 1523
Evaluating sample 1524
Evaluating sample 1525
Evaluating sample 1526
Evaluating 

...

Evaluating sample 8324
Evaluating sample 8325
Evaluating sample 8326
Evaluating sample 8327
Evaluating sample 8328
Evaluating sample 8329
Evaluating sample 8330
Evaluating sample 8331
Evaluating sample 8332
Evaluating sample 8333
Evaluating sample 8334
Evaluating sample 8335
Evaluating sample 8336
Evaluating sample 8337
Evaluating sample 8338
Evaluating sample 8339
Evaluating sample 8340
Evaluating sample 8341
Evaluating sample 8342
Evaluating sample 8343
Evaluating sample 8344
Evaluating sample 8345
Evaluating sample 8346
Evaluating sample 8347
Evaluating sample 8348
Evaluating sample 8349
Evaluating sample 8350
Evaluating sample 8351
Evaluating sample 8352
Evaluating sample 8353
Evaluating sample 8354
Evaluating sample 8355
Evaluating sample 8356
Evaluating sample 8357
Evaluating sample 8358
Evaluating sample 8359
Evaluating sample 8360
Evaluating sample 8361
Evaluating sample 8362
Evaluating sample 8363
Evaluating sample 8364
Evaluating sample 8365
Evaluating sample 8366
Evaluating 

Evaluating sample 8684
Evaluating sample 8685
Evaluating sample 8686
Evaluating sample 8687
Evaluating sample 8688
Evaluating sample 8689
Evaluating sample 8690
Evaluating sample 8691
Evaluating sample 8692
Evaluating sample 8693
Evaluating sample 8694
Evaluating sample 8695
Evaluating sample 8696
Evaluating sample 8697
Evaluating sample 8698
Evaluating sample 8699
Evaluating sample 8700
Evaluating sample 8701
Evaluating sample 8702
Evaluating sample 8703
Evaluating sample 8704
Evaluating sample 8705
Evaluating sample 8706
Evaluating sample 8707
Evaluating sample 8708
Evaluating sample 8709
Evaluating sample 8710
Evaluating sample 8711
Evaluating sample 8712
Evaluating sample 8713
Evaluating sample 8714
Evaluating sample 8715
Evaluating sample 8716
Evaluating sample 8717
Evaluating sample 8718
Evaluating sample 8719
Evaluating sample 8720
Evaluating sample 8721
Evaluating sample 8722
Evaluating sample 8723
Evaluating sample 8724
Evaluating sample 8725
Evaluating sample 8726
Evaluating 

Evaluating sample 9044
Evaluating sample 9045
Evaluating sample 9046
Evaluating sample 9047
Evaluating sample 9048
Evaluating sample 9049
Evaluating sample 9050
Evaluating sample 9051
Evaluating sample 9052
Evaluating sample 9053
Evaluating sample 9054
Evaluating sample 9055
Evaluating sample 9056
Evaluating sample 9057
Evaluating sample 9058
Evaluating sample 9059
Evaluating sample 9060
Evaluating sample 9061
Evaluating sample 9062
Evaluating sample 9063
Evaluating sample 9064
Evaluating sample 9065
Evaluating sample 9066
Evaluating sample 9067
Evaluating sample 9068
Evaluating sample 9069
Evaluating sample 9070
Evaluating sample 9071
Evaluating sample 9072
Evaluating sample 9073
Evaluating sample 9074
Evaluating sample 9075
Evaluating sample 9076
Evaluating sample 9077
Evaluating sample 9078
Evaluating sample 9079
Evaluating sample 9080
Evaluating sample 9081
Evaluating sample 9082
Evaluating sample 9083
Evaluating sample 9084
Evaluating sample 9085
Evaluating sample 9086
Evaluating 

Evaluating sample 9404
Evaluating sample 9405
Evaluating sample 9406
Evaluating sample 9407
Evaluating sample 9408
Evaluating sample 9409
Evaluating sample 9410
Evaluating sample 9411
Evaluating sample 9412
Evaluating sample 9413
Evaluating sample 9414
Evaluating sample 9415
Evaluating sample 9416
Evaluating sample 9417
Evaluating sample 9418
Evaluating sample 9419
Evaluating sample 9420
Evaluating sample 9421
Evaluating sample 9422
Evaluating sample 9423
Evaluating sample 9424
Evaluating sample 9425
Evaluating sample 9426
Evaluating sample 9427
Evaluating sample 9428
Evaluating sample 9429
Evaluating sample 9430
Evaluating sample 9431
Evaluating sample 9432
Evaluating sample 9433
Evaluating sample 9434
Evaluating sample 9435
Evaluating sample 9436
Evaluating sample 9437
Evaluating sample 9438
Evaluating sample 9439
Evaluating sample 9440
Evaluating sample 9441
Evaluating sample 9442
Evaluating sample 9443
Evaluating sample 9444
Evaluating sample 9445
Evaluating sample 9446
Evaluating 

Evaluating sample 9764
Evaluating sample 9765
Evaluating sample 9766
Evaluating sample 9767
Evaluating sample 9768
Evaluating sample 9769
Evaluating sample 9770
Evaluating sample 9771
Evaluating sample 9772
Evaluating sample 9773
Evaluating sample 9774
Evaluating sample 9775
Evaluating sample 9776
Evaluating sample 9777
Evaluating sample 9778
Evaluating sample 9779
Evaluating sample 9780
Evaluating sample 9781
Evaluating sample 9782
Evaluating sample 9783
Evaluating sample 9784
Evaluating sample 9785
Evaluating sample 9786
Evaluating sample 9787
Evaluating sample 9788
Evaluating sample 9789
Evaluating sample 9790
Evaluating sample 9791
Evaluating sample 9792
Evaluating sample 9793
Evaluating sample 9794
Evaluating sample 9795
Evaluating sample 9796
Evaluating sample 9797
Evaluating sample 9798
Evaluating sample 9799
Evaluating sample 9800
Evaluating sample 9801
Evaluating sample 9802
Evaluating sample 9803
Evaluating sample 9804
Evaluating sample 9805
Evaluating sample 9806
Evaluating 

In [ ]:
# This will generate PGD adversarial examples with L-infinity perturbation metric
# THIS WOULD BE TIME CONSUMING (Generating PGD adversarial examples of all 10000 CIFAR10 test samples)
# SoftMax vs REGroup Accuracy (CIFAR10 Test Set) on Adversarial Samples generated using PGD with L-Infinity perturbation Metric


pgd_linf_cifar10_samples = np.zeros((10000,3,32,32),np.float)
sam_id=0
adv_softmax_correct_pred = 0
adv_REGroup_correct_pred = 0
tot_samples = 0
all_effective_dist=[]
model.eval()
for cid in range(0,10,1):
    samples = glob.glob('./split_cifar10_data/'+split+'/'+str(cid)+'/*.mat')
    tot_samples = tot_samples+len(samples)
    for i in range(0,len(samples),1):
        print("Generating and Evaluating PGD Adv Examples of Class:{} samples:{}".format(cid,i))
        data = sio.loadmat(samples[i])
        img =  torch.tensor(data['img']).permute(1,2,0).numpy()
        input = Image.fromarray(img)
        input = transform(input).unsqueeze(0).to(device)
        
        image =  data['img'].astype(np.float32)#torch.tensor(data['img']).permute(1,2,0).numpy()
        image = image / 255.
        label = torch.tensor(data['label'])[0][0].to(device)
        
        #---------Clean Sample SoftMax Prediction--------------
        with torch.no_grad():
            output = model(input)

        smax = torch.softmax(output,dim=1)
        pred_class = torch.argmax(smax)
        
        # Generate adversarial example only if originally CORRECTLY classified by softmax
        if pred_class == label:
            adv_img, eff_adv_dist=RG_model.get_foolbox_pgd_linf_adversary(image,label.item(),epsilon=2.0)
            all_effective_dist.append(eff_adv_dist)
            adv_img = adv_img.unsqueeze(0)
            print(adv_img.shape)
        else:
            adv_img = input.clone()
        #------------------------------------------
        pgd_linf_cifar10_samples[sam_id,:,:,:] = adv_img[0,:,:,:].detach().cpu().numpy()
        sam_id=sam_id+1
        #---------Adversarial Sample SoftMax Prediction--------------
        with torch.no_grad():
            adv_output = model(adv_img)

        adv_smax = torch.softmax(adv_output,dim=1)
        adv_pred_class = torch.argmax(adv_smax)
        
        if adv_pred_class == label:
            adv_softmax_correct_pred = adv_softmax_correct_pred+1
            
        
        #--------REGroup Prediction----------------
        adv_REGroup_ranked_predictions=RG_model.get_robust_predictions(img=adv_img)
        adv_Regroup_pred_class = adv_REGroup_ranked_predictions[0]   #Top-1 Class

        if adv_Regroup_pred_class == label:
            adv_REGroup_correct_pred = adv_REGroup_correct_pred+1
        #------------------------------------------

SoftMax_CIFAR10_Test_Adv_Samples_ACC = (adv_softmax_correct_pred/tot_samples)*100
REGroup_CIFAR10_Test_Adv_Samples_ACC = (adv_REGroup_correct_pred/tot_samples)*100

print('CIFAR10 Test set average PGD Linfinity perturbation epsilon {}/256'.format(np.round(np.mean(all_effective_dist))))
print('Adversarial Samples Accuracy. PreTrained+SoftMax:{}%, PreTrained+REGroup:{}%'.format(SoftMax_CIFAR10_Test_Adv_Samples_ACC,
                                                                                  REGroup_CIFAR10_Test_Adv_Samples_ACC ))